In [17]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch
import numpy as np

In [39]:
data = pd.read_csv("2025-04-08/data_4.csv")
data

,x,y,RSSI_Sensor_1,RSSI_Sensor_2,RSSI_Sensor_3,RSSI_Sensor_4,Avg_RSSI
0,0.000000,0.0,-40.000000,-58.480892,-61.929538,-57.613008,-54.505860
1,0.129032,0.0,-13.467554,-57.903446,-62.903377,-56.727846,-47.750556
2,0.258065,0.0,-22.003460,-57.698459,-61.074211,-57.523698,-49.574957
3,0.387097,0.0,-27.204373,-58.092265,-62.736875,-56.672212,-51.176431
4,0.516129,0.0,-31.835391,-58.072074,-63.166538,-56.998134,-52.518034
...,...,...,...,...,...,...,...
1019,3.483871,4.0,-62.234700,-55.957500,-31.915694,-58.996333,-52.276057
1020,3.612903,4.0,-62.157096,-57.091609,-28.152887,-58.149972,-51.387891
1021,3.741935,4.0,-62.582892,-56.841414,-21.270734,-58.051512,-49.686638
1022,3.870968,4.0,-62.824829,-57.106038,-13.711246,-57.076840,-47.679738


In [40]:
X = data[["RSSI_Sensor_1", "RSSI_Sensor_2", "RSSI_Sensor_3", "RSSI_Sensor_4"]]
y = data[["x", "y"]]

scaler_X = StandardScaler().fit(X)
scaler_y = StandardScaler().fit(y)

X_scaled = scaler_X.transform(X)
y_scaled = scaler_y.transform(y)

In [41]:
class RSSIModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(4, 128),
            nn.ReLU(),
            nn.LayerNorm(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        return self.layers(x)


In [42]:
dataset = TensorDataset(torch.FloatTensor(X_scaled),
                       torch.FloatTensor(y_scaled))
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)

model = RSSIModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [43]:
for epoch in range(50):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')


Epoch 0, Loss: 0.8131
Epoch 0, Loss: 0.8390
Epoch 0, Loss: 0.5888
Epoch 0, Loss: 0.6205
Epoch 0, Loss: 0.4555
Epoch 0, Loss: 0.3492
Epoch 0, Loss: 0.3091
Epoch 0, Loss: 0.2508
Epoch 0, Loss: 0.1518
Epoch 0, Loss: 0.1764
Epoch 0, Loss: 0.1828
Epoch 0, Loss: 0.1326
Epoch 0, Loss: 0.1101
Epoch 0, Loss: 0.1282
Epoch 0, Loss: 0.1349
Epoch 0, Loss: 0.1145
Epoch 0, Loss: 0.1179
Epoch 0, Loss: 0.0863
Epoch 0, Loss: 0.0803
Epoch 0, Loss: 0.0987
Epoch 0, Loss: 0.1031
Epoch 0, Loss: 0.0894
Epoch 0, Loss: 0.0826
Epoch 0, Loss: 0.0892
Epoch 0, Loss: 0.0744
Epoch 0, Loss: 0.0768
Epoch 0, Loss: 0.0786
Epoch 0, Loss: 0.0508
Epoch 0, Loss: 0.0533
Epoch 0, Loss: 0.0620
Epoch 0, Loss: 0.0455
Epoch 0, Loss: 0.0492
Epoch 1, Loss: 0.0455
Epoch 1, Loss: 0.0415
Epoch 1, Loss: 0.0654
Epoch 1, Loss: 0.0313
Epoch 1, Loss: 0.0415
Epoch 1, Loss: 0.0448
Epoch 1, Loss: 0.0309
Epoch 1, Loss: 0.0391
Epoch 1, Loss: 0.0407
Epoch 1, Loss: 0.0285
Epoch 1, Loss: 0.0363
Epoch 1, Loss: 0.0409
Epoch 1, Loss: 0.0431
Epoch 1, L

In [44]:
with torch.no_grad():
    test_inputs = torch.FloatTensor(X_scaled)
    predictions = model(test_inputs)
    mse = criterion(predictions, torch.FloatTensor(y_scaled))
    print(f'Final MSE: {mse.item():.4f}')


Final MSE: 0.0076


In [45]:
predicted_coords = scaler_y.inverse_transform(predictions.numpy())
predicted_coords

array([[0.37913704, 0.25290608],
       [0.13474119, 0.0595299 ],
       [0.13241637, 0.17986596],
       ...,
       [3.7915642 , 3.8454747 ],
       [3.8846998 , 3.7915363 ],
       [3.65026   , 3.6309896 ]], dtype=float32)

In [46]:
input_rssi = np.array([[-58.45673382067411, -60.96910013008056, -54.31363764158987, -40.0]])

input_df = pd.DataFrame(
    input_rssi,
    columns=['RSSI_Sensor_1', 'RSSI_Sensor_2', 'RSSI_Sensor_3', 'RSSI_Sensor_4']
)

input_scaled = scaler_X.transform(input_df)

input_tensor = torch.FloatTensor(input_scaled)
with torch.no_grad():
    predicted = model(input_tensor)

coords = scaler_y.inverse_transform(predicted.numpy())

print(f"Координаты: x={coords[0][0]:.2f}, y={coords[0][1]:.2f}")

Координаты: x=3.83, y=1.05


In [29]:
data = pd.read_csv('tests/data_3.csv')

X = data[['RSSI_Sensor_1', 'RSSI_Sensor_2', 'RSSI_Sensor_3', 'RSSI_Sensor_4']]
y_actual = data[['x', 'y']]

X_scaled = scaler_X.transform(X)

model.eval()
with torch.no_grad():
    predictions = model(torch.FloatTensor(X_scaled))

y_pred = scaler_y.inverse_transform(predictions.numpy())

results = pd.DataFrame({
    'X_pred': y_pred[:,0],
    'Y_pred': y_pred[:,1],
    'X_actual': y_actual['x'],
    'Y_actual': y_actual['y']
})

results['X_error'] = abs(results['X_pred'] - results['X_actual'])
results['Y_error'] = abs(results['Y_pred'] - results['Y_actual'])
print(results)

mean_x_error = round(results['X_error'].mean(), 2)
mean_y_error = round(results['Y_error'].mean(), 2)

print(f"\nСредняя ошибка по X: {mean_x_error} мм")
print(f"Средняя ошибка по Y: {mean_y_error} мм")


      X_pred    Y_pred  X_actual  Y_actual   X_error   Y_error
0   0.059836  2.993924       0.0       3.0  0.059836  0.006076
1   0.127623  2.021500       0.0       2.0  0.127623  0.021500
2   0.901795  0.061312       1.0       0.0  0.098205  0.061312
3   2.026129  0.162497       2.0       0.0  0.026129  0.162497
4   3.118018  0.075357       3.0       0.0  0.118018  0.075357
5   3.927879  0.865852       4.0       1.0  0.072121  0.134148
6   3.943066  2.009057       4.0       2.0  0.056934  0.009057
7   0.921069  3.880468       1.0       4.0  0.078931  0.119532
8   3.171164  3.613953       3.0       3.5  0.171164  0.113953
9   3.171164  3.613953       3.0       3.5  0.171164  0.113953
10  1.528146  3.078018       1.5       3.0  0.028146  0.078018
11  2.039826  3.076720       2.0       3.0  0.039826  0.076720
12  1.473037  1.986397       1.5       2.0  0.026963  0.013603
13  3.171164  3.613953       3.0       3.5  0.171164  0.113953
14  1.968378  0.976873       2.0       1.0  0.031622  0

In [38]:
data = pd.read_csv('tests/data_3.csv')

X = data[['RSSI_Sensor_1', 'RSSI_Sensor_2', 'RSSI_Sensor_3', 'RSSI_Sensor_4']]
y_actual = data[['x', 'y']]

X_scaled = scaler_X.transform(X)

model.eval()
with torch.no_grad():
    predictions = model(torch.FloatTensor(X_scaled))

y_pred = scaler_y.inverse_transform(predictions.numpy())

results = pd.DataFrame({
    'X_pred': y_pred[:,0],
    'Y_pred': y_pred[:,1],
    'X_actual': y_actual['x'],
    'Y_actual': y_actual['y']
})

results['X_error'] = abs(results['X_pred'] - results['X_actual'])
results['Y_error'] = abs(results['Y_pred'] - results['Y_actual'])
print(results)

mean_x_error = round(results['X_error'].mean(), 2)
mean_y_error = round(results['Y_error'].mean(), 2)

print(f"\nСредняя ошибка по X: {mean_x_error} мм")
print(f"Средняя ошибка по Y: {mean_y_error} мм")

      X_pred    Y_pred  X_actual  Y_actual   X_error   Y_error
0   0.086710  3.157277       0.0       3.0  0.086710  0.157277
1   0.151081  2.011451       0.0       2.0  0.151081  0.011451
2   0.861473  0.125118       1.0       0.0  0.138527  0.125118
3   1.996171  0.110824       2.0       0.0  0.003829  0.110824
4   3.049212  0.134563       3.0       0.0  0.049212  0.134563
5   3.983589  0.950980       4.0       1.0  0.016411  0.049020
6   3.980097  2.065812       4.0       2.0  0.019903  0.065812
7   0.927718  3.928620       1.0       4.0  0.072282  0.071380
8   3.080781  3.605576       3.0       3.5  0.080781  0.105576
9   3.080781  3.605576       3.0       3.5  0.080781  0.105576
10  1.509490  3.047324       1.5       3.0  0.009490  0.047324
11  1.944068  3.004498       2.0       3.0  0.055932  0.004498
12  1.543664  2.023264       1.5       2.0  0.043664  0.023264
13  3.080781  3.605576       3.0       3.5  0.080781  0.105576
14  2.059221  0.898655       2.0       1.0  0.059221  0

In [47]:
data = pd.read_csv('tests/data_3.csv')

X = data[['RSSI_Sensor_1', 'RSSI_Sensor_2', 'RSSI_Sensor_3', 'RSSI_Sensor_4']]
y_actual = data[['x', 'y']]

X_scaled = scaler_X.transform(X)

model.eval()
with torch.no_grad():
    predictions = model(torch.FloatTensor(X_scaled))

y_pred = scaler_y.inverse_transform(predictions.numpy())

results = pd.DataFrame({
    'X_pred': y_pred[:,0],
    'Y_pred': y_pred[:,1],
    'X_actual': y_actual['x'],
    'Y_actual': y_actual['y']
})

results['X_error'] = abs(results['X_pred'] - results['X_actual'])
results['Y_error'] = abs(results['Y_pred'] - results['Y_actual'])
print(results)

mean_x_error = round(results['X_error'].mean(), 2)
mean_y_error = round(results['Y_error'].mean(), 2)

print(f"\nСредняя ошибка по X: {mean_x_error} мм")
print(f"Средняя ошибка по Y: {mean_y_error} мм")

      X_pred    Y_pred  X_actual  Y_actual   X_error   Y_error
0   0.031011  3.007160       0.0       3.0  0.031011  0.007160
1   0.022159  2.057177       0.0       2.0  0.022159  0.057177
2   0.925716  0.079357       1.0       0.0  0.074284  0.079357
3   1.926937  0.053517       2.0       0.0  0.073063  0.053517
4   2.945020  0.125599       3.0       0.0  0.054980  0.125599
5   3.827136  1.051115       4.0       1.0  0.172864  0.051115
6   3.841686  1.993421       4.0       2.0  0.158314  0.006579
7   0.952452  3.911612       1.0       4.0  0.047548  0.088388
8   3.040142  3.481707       3.0       3.5  0.040142  0.018293
9   3.040142  3.481707       3.0       3.5  0.040142  0.018293
10  1.478303  2.977706       1.5       3.0  0.021697  0.022294
11  1.980449  2.952385       2.0       3.0  0.019551  0.047615
12  1.543170  2.029578       1.5       2.0  0.043170  0.029578
13  3.040142  3.481707       3.0       3.5  0.040142  0.018293
14  1.935463  0.995211       2.0       1.0  0.064537  0